# Team Neural Network Challenge

This example is somewhat similiar to the classic MNIST number classificaiton project, but with a larger and potentially more interesting dataset. Details here: http://ufldl.stanford.edu/housenumbers/

Citation: Yuval Netzer, Tao Wang, Adam Coates, Alessandro Bissacco, Bo Wu, Andrew Y. Ng Reading Digits in Natural Images with Unsupervised Feature Learning NIPS Workshop on Deep Learning and Unsupervised Feature Learning 2011.

We will be using Keras to build a convolutional neural network and predict house numbers from pictures. 

## Importing Packages

In [1]:
# Reading in MATLAB files
import scipy.io 

# Preparing the Data
import numpy as np 
from PIL import ImageOps, Image # Equalise the image
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Building and Testing the Network
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


## Preparing the Data

In [3]:
training_input = scipy.io.loadmat('train_32x32.mat')

#Process Image Data
def process_image(image_array):
    
    img_in = Image.fromarray(image_array)
    img_equalised = ImageOps.equalize(img_in, mask=None)
    img_final = np.array(img_equalised, dtype=np.float32)
    
    return img_final

all_images = []

for i in range(73257):
    new_image = []
    for j in range(32):
        new_row = []
        for k in range(32):
            new_row.append([training_input['X'][j][k][0][i], training_input['X'][j][k][1][i], training_input['X'][j][k][2][i]])
        new_image.append(new_row)        
    all_images.append(process_image(np.asarray(new_image))/255)
    
all_images_array = np.asarray(all_images)

#Process Tags
all_tags = []
for i in range(73257):
    all_tags.append(training_input['y'][i][0])

all_tags_array = np.asarray(all_tags)

# Label encoding tags
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(all_tags_array.reshape(-1))
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

# One hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

X_train, X_test, y_train, y_test = train_test_split(all_images_array, onehot_encoded, test_size=0.20, random_state=42)

## Building the Neural Network

In [4]:
#This code is largely borrowed from Keras' MNIST example.

batch_size = 32
num_classes = 10
epochs = 2

img_rows, img_cols = 32, 32

x_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
x_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'Training samples')
print(x_test.shape[0], 'Testing samples')

x_train shape: (58605, 32, 32, 3)
y_train shape: (58605, 10)
58605 Training samples
14652 Testing samples


In [5]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=input_shape))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

Train on 58605 samples, validate on 14652 samples
Epoch 1/2
58605/58605 [==============================] - 32s 543us/step - loss: 1.8995 - acc: 0.3714 - val_loss: 1.7453 - val_acc: 0.4211
Epoch 2/2
58605/58605 [==============================] - 32s 540us/step - loss: 1.6696 - acc: 0.4612 - val_loss: 1.7080 - val_acc: 0.4421


## Testing the Neural Network

### Test Data Set

In [6]:
print("Testing on test data:")
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Testing on test data:
Test loss: 1.7080498358085772
Test accuracy: 0.4421239421320782
